# 1. Load the official dataset

In [1]:
# Load the official dataset
import pandas as pd

df_oficiales = pd.read_csv('/001/usuarios/juanmvs/anaconda3/NLP/Datos/DatosOficiales.csv')
df_oficiales = df_oficiales.drop(columns=['Unnamed: 0', 'Title', 'Place', 'Gender', 'Age', 'Country', 'Date'])
df_oficiales['Label'] = df_oficiales['Label'].replace([1,2,3,4,5], [0,1,2,3,4])
df_oficiales.shape

(5194, 2)

In [2]:
# Function for removing quotes 
def remove_quotes(dataframe):
    nw = []
    for op in dataframe['Opinion']:
        nw.append(op.strip('"'))
    dataframe['Opinion'] = nw

In [3]:
# Remove the quotes in the reviews
remove_quotes(df_oficiales)

In [4]:
# Check the max length of the reviews in the dataset
arr = []
for op in df_oficiales['Opinion']:
    mx = arr.append(len(op))
print(max(arr))

3769


In [5]:
# Split the reviews into longer and smaller than len(128)
longer128 = []
smaller128 = []

for op in df_oficiales['Opinion']:
    if(len(op)>128):
        longer128.append(op)
        
#for op in df0['Opinion']:
for op in df_oficiales['Opinion']:
    if(len(op)<=128):
        smaller128.append(op)

In [6]:
# Check how many reviews are shorter than 128 characters
print(len(smaller128), "reviews in the dataset are shorter or equal to 128 characters.")

964 reviews in the dataset are shorter or equal to 128 characters.


In [7]:
print(len(longer128), "reviews in the dataset are longer than 128 characters.")

4230 reviews in the dataset are longer than 128 characters.


In [8]:
# Split the reviews into chunks of (len)<=128
from io import StringIO

empty_list = []
for opinion in df_oficiales['Opinion']:
    string = StringIO(opinion)
    empty_list.append(list(iter(lambda: string.read(128), '')))

In [9]:
#flatten the 2d list 
flat = [j for sub in empty_list for j in sub]

In [10]:
# Create the new dataset. Each row contains a review <= 128 characters.
df = pd.DataFrame(flat, columns=['Opinion'])

In [11]:
num = df.shape[0]
print("The new DataFrame has", num, "reviews (one review per row). Each review has a length <= 128.")

The new DataFrame has 12182 reviews (one review per row). Each review has a length <= 128.


# 2. Adding the labels

In [12]:
# Check the distribution of the labels in the original dataset
df_oficiales['Label'].value_counts()

4    2688
3    1595
2     686
1     145
0      80
Name: Label, dtype: int64

In order to add the labels to the unlabeled chunks:
- First: Get the indexes where each label begins.
- Second: Create five lists, each one containing the values of the five labels. These lists will have a size proportional to the original dataset, but adjusting for the newly created dataset that contains reviews no longer than 128 characters.
- Finally: Paste those five lists in a new column of the DataFrame that contains the reviews with len <= 128.

* Note: This can be done as described above thanks to the fact that the reviews are ordered in ascending order to their labels. So, the rows go from label0 -> label1 -> ... -> label4.

In [13]:
# Label 1 begins at this index
df_oficiales['Opinion'][80] 

'Un callejón estrecho, sin más ni más, con """"guías""" \ninsistiendo en contarte la historia del callejón por una \nmódica propina, y el lugar a rebentar de gente esperando\npara hacer una foto imposible.'

In [14]:
# Label 2 begins at this index
df_oficiales['Opinion'][80+145] 

'Pasé varias veces por allí sin darme cuenta que este era el callejón. Lo más bonito es la historia, puesto que si bien el lugar es pintoresco, y los balcones parecen de un cuento de hadas, por como se lo ve en la ciudad parecería estar un poco sobredimensionado. De todas formas, no lleva mucho tiempo verlo, estando en Guanajuato hay que ir si o si.'

In [15]:
# Label 3 begins at this index
df_oficiales['Opinion'][80+145+686]

'Una visita tradicional, por la popularidad mundial que tienen, sin embargo hay que preguntarse ¿Qué es de entretenido visitar 22 cadáveres momificados? En lo personal y muy en lo personal es una experiencia diferente al imaginarse a estas personas encapsuladas en sus cuerpos con remanentes de épocas pasadas, nos dan una idea de la vida que en ese tiempo afrontaron y finalmente sucumbieron a ella, en mi opinión es un viaje de ida y nada más.'

In [16]:
# Label 4 begins at this index
df_oficiales['Opinion'][80+145+686+1595] 

'Callejón con historia, parada obligatoria si vas en pareja, \nademás de que es un muy bonito recuerdo'

In [17]:
# Get the index of the first review with label 1
df[df['Opinion'].str.match('Un callejón estrecho, sin más ni más')]

,Opinion
217,"Un callejón estrecho, sin más ni más, con """"""""..."


In [18]:
# Get the index of the first review with label 2
df[df['Opinion'].str.match('Pasé varias veces por allí sin darme cuenta que este era el callejón.')]

,Opinion
631,Pasé varias veces por allí sin darme cuenta qu...


In [19]:
# Get the index of the first review with label 3
df[df['Opinion'].str.match('Una visita tradicional, por la popularidad mundial que tienen, sin embargo hay que preguntarse ¿Qué es')]

,Opinion
2381,"Una visita tradicional, por la popularidad mun..."


In [20]:
# Get the index of the first review with label 4
df[df['Opinion'].str.match('Callejón con historia, parada ')]

,Opinion
6206,"Callejón con historia, parada obligatoria si v..."


In [21]:
# Create five lists of integers containing the labels that correspond to the chopped reviews
label0 = [0 for i in range(216)]
label1 = [1 for i in range(413)]
label2 = [2 for i in range(1750)]  
label3 = [3 for i in range(3825)]
label4 = [4 for i in range(5977)]

The following lines add the five lists one by one. This was done in order to check manually that the indexes were matching.

In [22]:
# add the list with the labels 0 
labels = label1.append(label2)

In [23]:
labelsx = [y for x in [label0, label1] for y in x]
len(labelsx)

630

In [24]:
labelsy = [y for x in [label2, label3] for y in x]
len(labelsy)

5575

In [25]:
labelsz = [y for x in [labelsx, labelsy] for y in x]
len(labelsz)

6205

In [26]:
labels = [y for x in [labelsz, label4] for y in x]
len(labels)

12182

In [27]:
# Finally add the list of labels to the newly created DataFrame 
df['Labels'] = labels
df

,Opinion,Labels
0,Las momias están en muy buen estado de conserv...,0
1,mi grupo quedó negativamente impresionado. Al...,0
2,on grabadas en lo profundo de las mentes de qu...,0
3,s con ganas de vomitar y espantados. Absolutam...,0
4,"do los muertos, los espantos y los entierros.....",0
...,...,...
12177,minada.,4
12178,"Es imperdible, de ahí puedes ver muy bien la c...",4
12179,hace parte de la aventura montarlo porque se ...,4
12180,Queda detrás del Teatro Juárez. Ir allí camina...,4


In [ ]:
# Export this DataFrame as a csv file
df.to_csv('Corpus128.csv')